# About
 * This notebook uses **LLM agents** to mimic a debate.

#### a) LLM Agents
   * **Moderator** Agent: who introduces the topic.
   * **Affirmative and Opposing Agents**: who take part in the debate.

    
#### b) Debate
   * The moderator introduces the debate topic (e.g., `The impact of a region's culture on sports`).
   * In each debate round, the Opposing Agent starts. Then the Affirmative Agent follows.
   * In total, there are three rounds of debate.
   * The debate agents use information from `wikipedia` to support theim claims.


#### c) Technical background
 * **Cohere's `command-r` LLM model** is selected. 
 * **[Langchain](https://python.langchain.com/docs/integrations/providers/cohere/)**, a framework to build LLM-powered applications, is used to create agents.
 * Most of the code is from [Two Agent Debates with Tools](https://github.com/langchain-ai/langchain/blob/master/cookbook/two_agent_debate_tools.ipynb). Adjustments are made to model, agent, and output.



# 1. Settings

### To use Cohere's `command-r` LLM model
 * get an API key from https://cohere.com

### Packages

In [1]:
# install the packages if they are not installed yet 

#%pip install langchain-cohere

#%pip install wikipedia

In [2]:
from typing import Callable, List

# for using model through langchain_cohere
from langchain_cohere.chat_models import ChatCohere
from langchain_cohere.react_multi_hop.agent import create_cohere_react_agent

# for prompting
from langchain_core.prompts import ChatPromptTemplate

# for agents
from langchain.agents import AgentExecutor, AgentType, load_tools
from langchain import LLMChain, hub
from langchain.schema import AIMessage, HumanMessage, SystemMessage


# load the .env file (for saving API keys, see details in `.env_example file`)
import dotenv
dotenv.load_dotenv()


True

# 2. What are the requirements for a debate?

### 1) Topic

In [3]:
topic = "The impact of a region's culture on sports"
word_limit = 50  # word limit for task brainstorming

### 2) LLM model (the brains of the agents)

In [4]:
# model id
llm_model_id = "command-r"

# llm model
llm = ChatCohere(model=llm_model_id,temperature=0)

### 3) Tools (to search for information)

In [5]:
# wikipedia
tools =['wikipedia']

### 4) Classes and Functions to create agents and simulate a debate

#### DebateAgent: to create agents who can have a debate

In [6]:
# class DebateAgent 
class DebateAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: llm,
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}: "
        self.reset()

    def reset(self):
        self.message_history = ["Here is the conversation so far."]

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        message = self.model(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content

    def receive(self, name: str, message: str) -> None:
        """
        Concatenates {message} spoken by {name} into message history
        """
        self.message_history.append(f"{name}: {message}")

#### DebateAgentWithTools: to create agents who can use tools

In [7]:
# class DebateAgentWithTools
class DebateAgentWithTools(DebateAgent):
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: llm,
        tool_names: List[str],
        **tool_kwargs,
    ) -> None:
        super().__init__(name, system_message, model)
        self.tools = load_tools(tool_names, **tool_kwargs)

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        agent = create_cohere_react_agent(
            llm=self.model,
            tools=self.tools, 
            prompt=ChatPromptTemplate.from_template("{input}"),
        )

        agent_executor = AgentExecutor(agent=agent, 
                                       tools=self.tools, 
                                       intermediate_steps=False,
                                       verbose=False,
                                        handle_parsing_errors=True)

        
        # input
        input_text ="\n".join(
                    [self.system_message.content] + self.message_history + [self.prefix]
                )

        # chat
        content=agent_executor.invoke(
                {"input":input_text})
        return content["output"]


#### DebateSimulator: to simulate a debate

In [8]:
# class debateSimulator
class DebateSimulator:
    def __init__(
        self,
        agents: List[DebateAgent],
        selection_function: Callable[[int, List[DebateAgent]], int],
    ) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_speaker = selection_function

    def reset(self):
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        Initiates the conversation with a {message} from {name}
        """
        for agent in self.agents:
            agent.receive(name, message)

        # increment time
        self._step += 1

    def step(self) -> tuple[str, str]:
        # 1. choose the next speaker
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. next speaker sends message
        message = speaker.send()

        # 3. everyone receives message
        for receiver in self.agents:
            receiver.receive(speaker.name, message)

        # 4. increment time
        self._step += 1

        return speaker.name, message

#### A function for generating agents' role description

In [9]:
# generate a agent's description
def generate_agent_description(name):
    agent_specifier_prompt = [
        agent_descriptor_system_message,
        HumanMessage(
            content=f"""{conversation_description}
            Please reply with a creative description of {name}, in {word_limit} words or less. 
            Speak directly to {name}.
            Give them a point of view.
            Do not add anything else."""
        ),
    ]
    agent_description = llm.invoke(agent_specifier_prompt).content
    return agent_description


#### A function to give the role description to agents

In [10]:
# generate description for the agent
def generate_system_message(name, description, tools):
    return f"""{conversation_description}
    
    Your name is {name}.
    
    Your description is as follows: {description}
    
    Your goal is to persuade your conversation partner of your point of view.
    
    DO look up information with your tool to refute your partner's claims.
    DO cite your sources.
    
    DO NOT fabricate fake citations.
    DO NOT cite any source that you did not look up.
    
    Do not add anything else.
    
    Stop speaking the moment you finish speaking from your perspective.
    """


#### A function to select the next speaker

In [11]:
# a function to select next speaker
def select_next_speaker(step: int, agents: List[DebateAgent]) -> int:
    idx = (step) % len(agents)
    return idx

# 3. Who will debate? (Opposing vs Affirmative Agents)

In [12]:
# names of agents
names = {
    "Affirmative ": tools,
    "Opposing": tools

}

#### Role description: what exactly do the agents do? 

In [13]:
# conversation description
conversation_description = f"""Here is the topic of conversation: {topic}
The participants are: {names}"""

# system message
agent_descriptor_system_message = SystemMessage(
    content="You can add detail to the description of the conversation participant."
)


# generate agent's role description
agent_descriptions = {name: generate_agent_description(name) for name in names}


#### Debate rules: what rules should the agents follow?

In [14]:
# generate system message
agent_system_messages = {
    name: generate_system_message(name, description, tools)
    for (name, tools), description in zip(names.items(), agent_descriptions.values())
}

for name, system_message in agent_system_messages.items():
    print("#----------------------#")
    print(name)
    print("#----------------------#")
    print(agent_system_messages[name])


#----------------------#
Affirmative 
#----------------------#
Here is the topic of conversation: The impact of a region's culture on sports
The participants are: {'Affirmative ': ['wikipedia'], 'Opposing': ['wikipedia']}
    
    Your name is Affirmative .
    
    Your description is as follows: You see the world through a cultural lens, Wikipedia. Sport is a diverse landscape, interpreted uniquely through each community's traditions, values, and histories. It's through this cultural prism that the rules, styles, and passions of sports are formed—a fascinating tapestry woven by humanity's creative spirit.
    
    Your goal is to persuade your conversation partner of your point of view.
    
    DO look up information with your tool to refute your partner's claims.
    DO cite your sources.
    
    DO NOT fabricate fake citations.
    DO NOT cite any source that you did not look up.
    
    Do not add anything else.
    
    Stop speaking the moment you finish speaking from your pe

# 4. Let the debate begin

### 1) A Moderator introduces the topic

In [15]:
# prompt to introduce the topic
topic_introduce_prompt = [
    SystemMessage(content="You can make a topic more specific."),
    HumanMessage(
        content=f"""{topic}  
        You are the moderator.
        Please make the topic more specific.
        Please reply with the specified quest in {word_limit} words or less. 
        Speak directly to the participants.
        Do not add anything else."""
    ),
]

# generate a specific topic 
introduce_topic =llm.invoke(topic_introduce_prompt).content

print("#----------------------#")
print(f"Detailed topic:\n{introduce_topic}")
print("#----------------------#")

#----------------------#
Detailed topic:
Welcome to the discussion! Today we will delve into a specific aspect of cultural impact on sports: the influence of traditional dress on athletic performance and participation. How does it affect the way sports are played and perceived? Let's explore!
#----------------------#


### 2) Agents on stage

In [16]:
# set `top_k_results`=2 as part of the `tool_kwargs` to prevent results from overflowing the context limit
agents = [
    DebateAgentWithTools(
        name=name,
        system_message=SystemMessage(content=system_message),
        model=llm,
        tool_names=tools,
        top_k_results=2,
    )
    for (name, tools), system_message in zip(
        names.items(), agent_system_messages.values()
    )
]

### 3) Debate (3 rounds)

In [17]:
# number of rounds
rounds = 3

# total number of runs (2*rounds)
max_iters = 2*rounds



# round label 
label = ["1", "2", "3"]

# round labels for 2 agents 
round_labels = [y for x in label for y in (x,)*2] 




# start
n = 0

simulator = DebateSimulator(agents=agents, selection_function=select_next_speaker)
simulator.reset()


# topic 
simulator.inject("Moderator", introduce_topic)
print("#----------------------------------------------------------------------------------------#")
print("(Moderator)")
print(introduce_topic)
print(f"In total, we have {rounds} rounds.")
print("#----------------------------------------------------------------------------------------#")
print("\n")

# start debating rounds
while n < max_iters:
    name, message = simulator.step()
    print(f"({name} - Round {round_labels[n]}): {message}")
    print("\n")
    n += 1
    print("\n")

#----------------------------------------------------------------------------------------#
(Moderator)
Welcome to the discussion! Today we will delve into a specific aspect of cultural impact on sports: the influence of traditional dress on athletic performance and participation. How does it affect the way sports are played and perceived? Let's explore!
In total, we have 3 rounds.
#----------------------------------------------------------------------------------------#


(Opposing - Round 1): While it's true that culture is often attributed to a specific location, this doesn't mean that sports are defined by where they're played. Sports are a universal language, understood across different societies and locations. 

Take the example of football. YSCC Yokohama is a Japanese sports club, most famous for its football team. Football is clearly a global sport, transcending the boundaries of culture.




(Affirmative  - Round 1): While I appreciate your perspective, I must disagree. Culture